In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from tqdm import tqdm
from rdkit import Chem as rdChem
from collections import OrderedDict
from copy import deepcopy
from source.utils.conforge_conformer_generation import generate_conformers, get_conformer_generator, rdkit_generate_conformers
from source.utils.mol2pyg import mols2pyg_list_with_targets, pyg2mol
from source.utils.mol_utils import get_energy, smi_reader_params, smi_writer_params, drop_disconnected_components,visualize_3d_mols, set_coords, preprocess_mol,has_steric_clashes,fix_conformer,minimum_atom_distance, get_rdkit_conformer
from source.utils.data_splitting_utils import create_data_folders, parse_smiles_file, get_smiles_and_targets_from_csv
from source.data_transforms._frad_transforms import frad
from source.utils.file_handling_utils import ls
from source.utils.npz_utils import get_field_from_npzs, save_npz
from source.utils.code_utils import print_obj_API

In [34]:
input_data = '/home/nobilm@usi.ch/pretrain_paper/data/halicin_data.csv'
smiles,targets = get_smiles_and_targets_from_csv(input_data)
max_confs = 1
n_confs_to_keep = 1


idx = 1501
smiles,targets = [smiles[idx]],[targets[idx]]

conf_generator = get_conformer_generator()
for s,y in tqdm(zip(smiles, targets), total=len(smiles)):
    s = drop_disconnected_components(s)
    conformers = generate_conformers(s, conf_generator)
    if not conformers: continue
    mol2pyg_kwargs = {"max_energy": max((get_energy(m) for m in conformers))}
    pyg_mol_confs = mols2pyg_list_with_targets(conformers, [s]*len(conformers), [y]*len(conformers), **mol2pyg_kwargs)
    dc_pyg_mol_confs = deepcopy(pyg_mol_confs)
    out_dihedrals_only = frad(pyg_mol_confs[0], add_coords_noise=False)
    out = frad(dc_pyg_mol_confs[0])

d = OrderedDict()
# d["rdkit_conf"]=rdkit_generate_conformers(preprocess_mol(rdChem.MolFromSmiles(smiles[0], smi_reader_params())),num_conformers=1)[0]
d["conforge"]=conformers[0]
d["dih"]=pyg2mol(out_dihedrals_only)
d["dih+cords"]=pyg2mol(out)

mols,titles = [],[]
for k,v in d.items():
    mols.append(v)
    titles.append(f"{k}:{get_energy(v):.1f}")

visualize_3d_mols(mols,titles=titles, width=1800, height=400)

  0%|          | 0/1 [00:00<?, ?it/s]/home/nobilm@usi.ch/pretrain_paper/source/data_transforms/_frad_transforms.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  angle_diff_rad = torch.deg2rad(torch.tensor(desired_angle_deg - current_angle_deg))
/home/nobilm@usi.ch/pretrain_paper/source/data_transforms/_frad_transforms.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  neighbors = torch.nonzero(torch.tensor(adjacency_matrix[current]), as_tuple=False).flatten()
100%|██████████| 1/1 [00:00<00:00, 63.55it/s]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [32]:
out.rotable_bonds

tensor([[ 0,  1,  2, 28],
        [ 4,  3,  1,  0],
        [ 8,  7,  6,  5],
        [11, 10,  9,  8],
        [10, 15, 16, 35],
        [20, 19, 18,  7],
        [19, 24, 25, 40]])

In [13]:
# input_data = '/home/nobilm@usi.ch/pretrain_paper/data/halicin_data.csv'
# smiles,targets = get_smiles_and_targets_from_csv(input_data)
# max_confs = 1
# n_confs_to_keep = 1
# idx = 2
# smiles,targets = [smiles[idx]],[targets[idx]]
# conf_generator = get_conformer_generator(max_confs=max_confs)
# smiles = [drop_disconnected_components(s) for s in smiles]

# for s,y in tqdm(zip(smiles, targets), total=len(smiles)):
#   conformers = generate_conformers(s, conf_generator, n_confs_to_keep)
#   mol2pyg_kwargs = {"max_energy": max((get_energy(m) for m in conformers))}
#   pyg_mol_confs = mols2pyg_list_with_targets(conformers, [s]*len(conformers), [y]*len(conformers), **mol2pyg_kwargs)
#   out_dihedrals_only = frad(pyg_mol_confs[0], add_coords_noise=False)
#   out = frad(pyg_mol_confs[0]) #, add_coords_noise=False)
# d = OrderedDict()
# d["rdkit_conf"]=rdkit_generate_conformers(preprocess_mol(rdChem.MolFromSmiles(smiles[0], smi_reader_params())),num_conformers=1)[0]
# d["conforge"]=conformers[0]
# d["dih"]=pyg2mol(out_dihedrals_only)
# d["dih+cords"]=pyg2mol(out)

# mols,titles = [],[]
# for k,v in d.items():
#   mols.append(v)
#   titles.append(f"{k}:{get_energy(v):.1f}")

# visualize_3d_mols(mols,titles=titles, width=2200, height=500)

# test read write npz

In [7]:
d = get_field_from_npzs('/storage_common/nobilm/pretrain_paper/guacamol/all/mol_1.npz')[0]
for k,v in d.items():
  print(k,v.shape)

d['smiles'], d['max_energy']

# pyg_mol_confs[0]['y']

coords (2, 45, 3)
atom_types (45,)
group (45,)
period (45,)
hybridization (45,)
chirality (45,)
is_aromatic (45,)
is_in_ring (45,)
smiles ()
adj_matrix (45, 45)
max_energy ()
graph_labels ()


(array('COc1ccc(OC)c2c1c(OC)cc1c(=O)cc(-c3ccccc3)oc12', dtype='<U45'),
 array(114.17016, dtype=float32))

# TEST FIX CLASHES

In [ ]:
input_data = '/home/nobilm@usi.ch/pretrain_paper/data/halicin_data.csv'
smiles,targets = get_smiles_and_targets_from_csv(input_data)
max_confs = 1
n_confs_to_keep = 1
idx = 2
smiles,targets = [smiles[idx]],[targets[idx]]


conf_generator = get_conformer_generator(max_confs=max_confs)
smiles = [drop_disconnected_components(s) for s in smiles]

for s,y in tqdm(zip(smiles, targets), total=len(smiles)):
  conformers = generate_conformers(s, conf_generator, n_confs_to_keep)
  mol2pyg_kwargs = {"max_energy": max((get_energy(m) for m in conformers))}
  pyg_mol_confs = mols2pyg_list_with_targets(conformers, [s]*len(conformers), [y]*len(conformers), **mol2pyg_kwargs)
  dc_pyg_mol_confs = deepcopy(pyg_mol_confs)
  out_dihedrals_only = frad(pyg_mol_confs[0], add_coords_noise=False)
  out = frad(dc_pyg_mol_confs[0]) #, add_coords_noise=False)

d = OrderedDict()
d["rdkit_conf"]=rdkit_generate_conformers(preprocess_mol(rdChem.MolFromSmiles(smiles[0], smi_reader_params())),num_conformers=1)[0]
d["conforge"]=conformers[0]
d["dih"]=pyg2mol(out_dihedrals_only)
d["dih+cords"]=pyg2mol(out)

mols,titles = [],[]
for k,v in d.items():
  mols.append(v)
  titles.append(f"{k}:{get_energy(v):.1f}")

visualize_3d_mols(mols,titles=titles, width=2200, height=500)

In [ ]:
# minimum_atom_distance(d["dih"]), minimum_atom_distance_vectorized(d["dih"]), minimum_atom_distance(d["dih"]) == minimum_atom_distance_vectorized(d["dih"])

In [ ]:
m_old = d["dih"]
mol = deepcopy(d["dih"])
print("Min dist before fix:", minimum_atom_distance(m_old))

m_new = fix_conformer(mol)
print("Min dist post fix:", minimum_atom_distance(m_new))

d = OrderedDict()
d["dih"]=m_old
d["updated"]=m_new

mols,titles = [],[]
for k,v in d.items():
  mols.append(v)
  titles.append(f"{k}:{get_energy(v):.1f}")

visualize_3d_mols(mols,titles=titles, width=2200, height=500)

# test execution times:


In [ ]:
import timeit

input_data = '/home/nobilm@usi.ch/pretrain_paper/data/halicin_data.csv'
smiles,targets = get_smiles_and_targets_from_csv(input_data)
mols = [preprocess_mol(rdChem.MolFromSmiles(smi, smi_reader_params())) for smi in smiles]
mols = [m for m in mols if get_rdkit_conformer(m)]

# Benchmark the execution time of line1
def benchmark_line1():
    for mol in mols: minimum_atom_distance(mol) >= 0.8

# Benchmark the execution time of line2
def benchmark_line2():
    for mol in mols: get_energy(mol) <= 2 * 10000

# Run the benchmarks
line1_time = timeit.timeit(benchmark_line1, number=100)
print(f"Line 1 execution time: {line1_time:.4f} seconds")
line2_time = timeit.timeit(benchmark_line2, number=100)
print(f"Line 2 execution time: {line2_time:.4f} seconds")



# TEST WRITE

In [ ]:
# # get halicin smiles + labels
# input_data = '/home/nobilm@usi.ch/pretrain_paper/data/halicin_data.csv'
# smiles,targets = get_smiles_and_targets_from_csv(input_data)

# # create dir
# dir = '/storage_common/nobilm/pretrain_paper/halicin_new_data_enc'
# all_dir, train_dir, val_dir, test_dir = create_data_folders(dir)
# N = 2
# conf_generator = get_conformer_generator(max_confs=5)
# i_save = 0
# for s,y in tqdm(zip(smiles, targets), total=len(smiles)):
#   conformers = generate_conformers(s, conf_generator, N)
#   mol2pyg_kwargs = {"max_energy": max(get_energy(m) for m in conformers)}
#   pyg_mol_confs = mols2pyg_list_with_targets(conformers, [y]*len(conformers), **mol2pyg_kwargs)
#   i_save = save_npz(pyg_mol_confs, folder_name=all_dir, idx=i_save)

# # check that final npzs must be < 25*nsmiles

# assert len(ls(all_dir)) <= len(smiles)*N

In [ ]:
# len(ls(all_dir)) # N=25*max_confs=1000=

# Viz Tests

In [ ]:
visualize_3d_mols(conformers)

# ConfGen exploration

In [ ]:
# /home/marconobile/CDPKit/CDPKit/Apps/CmdLine/ConfGen/ConfGenImpl.cpp
# https://cdpkit.org/v1.1.1/applications/confgen.html
# https://cdpkit.org/cdpl_api_doc/python_api_doc/classCDPL_1_1ConfGen_1_1ConformerGeneratorSettings.html#a8f00604dec7d4da5376bcd28b2051f42

In [ ]:
import CDPL.Chem as Chem
import CDPL.ConfGen as ConfGen
import CDPL.Chem as CDPLChem
from source.utils.code_utils import print_obj_API
conf_generator = ConfGen.ConformerGenerator()
conf_generator.settings.setEnergyWindow(20.0)
conf_generator.settings.setMaxNumOutputConformers(250)
conf_generator.settings.setMinRMSD(0.5)

print("getEnergyWindow", conf_generator.settings.getEnergyWindow())
print("getMaxNumOutputConformers", conf_generator.settings.getMaxNumOutputConformers())
print("getMaxNumSampledConformers", conf_generator.settings.getMaxNumSampledConformers())
print("getMinRMSD", conf_generator.settings.getMinRMSD())

In [ ]:
conf_generator = ConfGen.ConformerGenerator()
conf_generator.settings.setEnergyWindow(20.0)
conf_generator.settings.setMaxNumOutputConformers(250)
conf_generator.settings.setMinRMSD(0.5)
print("getEnergyWindow", conf_generator.settings.getEnergyWindow())
print("getMaxNumOutputConformers", conf_generator.settings.getMaxNumOutputConformers())
print("getMaxNumSampledConformers", conf_generator.settings.getMaxNumSampledConformers())
print("getMinRMSD", conf_generator.settings.getMinRMSD())

In [ ]:
print_obj_API(conf_generator.settings)

In [ ]:
conf_generator.settings.getSamplingMode()

In [ ]:
# Conformer generation preset to use: -C [ –conf-gen-preset ] arg
# (SMALL_SET_DIVERSE, MEDIUM_SET_DIVERSE, LARGE_SET_DIVERSE, SMALL_SET_DENSE, MEDIUM_SET_DENSE, LARGE_SET_DENSE, default: MEDIUM_SET_DIVERSE).

# -m [ –mode ] arg
# Conformer sampling mode (AUTO, STOCHASTIC, SYSTEMATIC, default: AUTO).

# -e [ –e-window ] arg
# Energy window for generated conformers.
# The energy window may be specified as a single constant value or  as a list of pairs RBC1 EW1 RBC2 EW2…
# where RBC denotes a rotatable bond count and EW the energy window that applies if the rotatable bond count of the processed molecule is <= RBC
# (the EW value associated with the lowest RBC that fulfills the latter condition takes precedence). If the rotatable bond count of the processed
# molecule is outside any defined range then the EW value associated with the highest RBC will be used. (default: 15.0, energy window values must be >= 0).

# -r [ –rmsd ] arg
# Minimum RMSD for output conformer selection.
# The RMSD may be specified as a single constant value or  as a list of pairs RBC1 RMSD1 RBC2 RMSD2…
# where RBC denotes a rotatable bond count and RMSD is the value that applies if the rotatable bond count of the processed molecule is <= RBC
# (the RMSD value associated with the lowest RBC that fulfills the latter condition takes precedence).
# If the rotatable bond count of the processed molecule is outside any defined range then the RMSD value associated with the highest RBC will be used.
# (default: 0.5, RMSD values must be >= 0, 0 disables RMSD checking).

# -n [ –max-num-out-confs ] arg
# Maximum number of output conformers per molecule. The max. number of output conformers may be specified as a single constant value or as a list of
# pairs RBC1 MC1 RBC2 MC2… where RBC denotes a rotatable bond count and MC the max. number of conformers that applies if the rotatable bond count of
# the processed molecule is <= RBC (the MC value associated with the lowest RBC that fulfills the latter condition takes precedence).
# If the rotatable bond count of the processed molecule is outside any defined range then the MC value associated with the highest RBC will be used.
# (default: 100, count values must be >= 0, 0 disables limit).

# -x [ –max-num-sampled-confs ] arg
# Maximum number of sampled conformers (only effective in stochastic sampling mode, default: 2000, must be >= 0, 0 disables limit).

# -y [ –conv-check-cycle-size ] arg
# Minimum number of duplicate conformers that have to be generated in succession to consider convergence to be reached
# (only effective in stochastic sampling mode, default: 100, must be > 0).

# -B [ –frag-build-preset ] arg
# Fragment build preset to use (FAST, THOROUGH, only effective in systematic sampling mode, default: FAST).

In [ ]:
# ConformerGeneratorSettings Attributes: [

#   'DEFAULT',

#   'LARGE_SET_DENSE',
#   'LARGE_SET_DIVERSE',
#   'MEDIUM_SET_DENSE',
#   'MEDIUM_SET_DIVERSE',
#   'SMALL_SET_DENSE',
#   'SMALL_SET_DIVERSE',

#   'samplingMode',
  # settings.setSamplingMode(ConformerSamplingMode::AUTO);
  # settings.setSamplingMode(ConformerSamplingMode::SYSTEMATIC);
  # settings.setSamplingMode(ConformerSamplingMode::STOCHASTIC);


#   'energyWindow',
#   'distanceExponent',
#   'fragmentBuildSettings',
#   'maxNumRefinementIterations',
#   'maxNumSampledConformers',
#   'maxPoolSize',
#   'maxRotatableBondCount',
#   'minRMSD',
#   'maxNumOutputConformers',
#   'refinementTolerance',
#   'strictForceFieldParam'


#   'forceFieldTypeSystematic',
#   'forceFieldTypeStochastic',

#   'nitrogenEnumMode',
#   'convCheckCycleSize',
#   'dielectricConstant',
#   'enumRings',
#   'genCoordsFromScratch',
#   'includeInputCoords',
#   'macrocycleRotorBondCountThresh',
#   'objectID',
#   'sampleAngleTolRanges',
#   'sampleHetAtomHydrogens',

#   'timeout'


# Test dis dihedrals sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
scale = 40
s = np.random.normal(0, scale, size=1000000)
s = np.clip(s, a_min=-180., a_max=180)
print(min(s), max(s), np.std(s))
plt.hist(s, bins=1000)
plt.show()

In [ ]:
scale = 40
s = np.random.standard_cauchy(size=1000000) * scale
s = np.clip(s, a_min=-180., a_max=180)
print(min(s), max(s), np.std(s))
plt.hist(s, bins=100)
plt.show()

# Test atom enc

In [7]:
from mendeleev.fetch import fetch_table
# from rdkit import Chem
# import numpy as np
# # pip install mendeleev

ptable = fetch_table("elements")
PATOMIC_NUMBERS = {row["symbol"]: row["atomic_number"] for _, row in ptable.iterrows()}
PGROUP_IDS = {row["symbol"]: row["group_id"] for _, row in ptable.iterrows()}
PPERIOD_IDS = {row["symbol"]: row["period"] for _, row in ptable.iterrows()}

len(PGROUP_IDS), len(PPERIOD_IDS)

(118, 118)

# test new pos

In [10]:
import torch

bs = 4  # example batch size
N = 5   # example value for N

out = []
for i in range(bs):
    l = []
    for j in range(N):
        l.append((1, 2, 3.0))
    out.append(torch.tensor(l))

# Stack the list of tensors into a single tensor with shape (bs, N, 3)
out_tensor = torch.stack(out)
print(out_tensor.shape)  # Should print torch.Size([bs, N, 3])


torch.Size([4, 5, 3])


In [9]:
# mutliprocess
import multiprocessing as mp
import torch
from tqdm import tqdm

def worker(smiles_target, conf_generator, n_confs_to_keep, all_dir, lock, shared_i_save):
    s, y = smiles_target
    s = drop_disconnected_components(s)
    conformers = generate_conformers(s, conf_generator, n_confs_to_keep)
    if not conformers:
        return

    mol2pyg_kwargs = {"max_energy": max((get_energy(m) for m in conformers))}
    pyg_mol_fixed_fields = mols2pyg_list_with_targets([conformers[0]], [s], [y], **mol2pyg_kwargs)[0]

    batched_pos = []
    for mol in conformers:
        pos = []
        conf = mol.GetConformer()
        for i, atom in enumerate(mol.GetAtoms()):
            positions = conf.GetAtomPosition(i)
            pos.append((positions.x, positions.y, positions.z))
        pos = torch.tensor(pos, dtype=torch.float32)
        batched_pos.append(pos)

    batched_pos = torch.stack(batched_pos)
    pyg_mol_fixed_fields.pos = batched_pos

    with lock:
        i_save = shared_i_save.value
        save_pyg_as_npz(pyg_mol_fixed_fields, f'{all_dir}/mol_{i_save}')
        shared_i_save.value += 1

def main(smiles, targets, conf_generator, n_confs_to_keep, all_dir):
    manager = mp.Manager()
    lock = manager.Lock()
    shared_i_save = manager.Value('i', 0)

    with mp.Pool(mp.cpu_count()) as pool:
        tasks = [(s, y) for s, y in zip(smiles, targets)]
        results = list(tqdm(pool.imap_unordered(worker, tasks), total=len(smiles)))

if __name__ == "__main__":
    # Initialize your variables `smiles`, `targets`, `conf_generator`, `n_confs_to_keep`, `all_dir` here
    main(smiles, targets, conf_generator, n_confs_to_keep, all_dir)


AttributeError: 'list' object has no attribute 'shape'

In [12]:
import CDPL.Chem as Chem
import CDPL.ConfGen as ConfGen
import CDPL.Chem as CDPLChem
conf_gen = ConfGen.ConformerGenerator()


In [16]:
print_obj_API(conf_gen.settings)

ConformerGeneratorSettings Attributes: ['DEFAULT', 'LARGE_SET_DENSE', 'LARGE_SET_DIVERSE', 'MEDIUM_SET_DENSE', 'MEDIUM_SET_DIVERSE', 'SMALL_SET_DENSE', 'SMALL_SET_DIVERSE', 'convCheckCycleSize', 'dielectricConstant', 'distanceExponent', 'energyWindow', 'enumRings', 'forceFieldTypeStochastic', 'forceFieldTypeSystematic', 'fragmentBuildSettings', 'genCoordsFromScratch', 'includeInputCoords', 'macrocycleRotorBondCountThresh', 'maxNumOutputConformers', 'maxNumRefinementIterations', 'maxNumSampledConformers', 'maxPoolSize', 'maxRotatableBondCount', 'minRMSD', 'nitrogenEnumMode', 'objectID', 'refinementTolerance', 'sampleAngleTolRanges', 'sampleHetAtomHydrogens', 'samplingMode', 'strictForceFieldParam', 'timeout']


ConformerGeneratorSettings Methods: ['addEnergyWindowRange', 'addMaxNumOutputConformersRange', 'addMinRMSDRange', 'assign', 'clearEnergyWindowRanges', 'clearMaxNumOutputConformersRanges', 'clearMinRMSDRanges', 'enumerateRings', 'generateCoordinatesFromScratch', 'getConvergenceChe

In [18]:
from source.utils.npz_utils import get_field_from_npzs
d = get_field_from_npzs('/storage_common/nobilm/pretrain_paper/second_test/all/mol_0.npz')[0]
for k,v in d.items():
  print(k,v.shape)


coords (2, 24, 3)
atom_types (24,)
group (24,)
period (24,)
graph_labels ()
hybridization (24,)
chirality (24,)
is_aromatic (24,)
is_in_ring (24,)
smiles ()
rotable_bonds (3, 4)
dihedral_angles_degrees (3,)


In [20]:
d['coords'][0]

array([[ 4.1532e+00, -1.5440e-01,  1.1000e-02],
       [ 3.3906e+00, -1.2998e+00,  7.5000e-03],
       [ 3.9976e+00, -2.5581e+00, -8.2000e-03],
       [ 3.2271e+00, -3.7303e+00, -1.0900e-02],
       [ 4.0777e+00, -5.2262e+00, -3.9100e-02],
       [ 1.8242e+00, -3.6951e+00, -1.1000e-03],
       [ 1.2470e+00, -2.4106e+00,  0.0000e+00],
       [ 1.9982e+00, -1.2273e+00,  0.0000e+00],
       [ 1.3760e+00,  0.0000e+00,  0.0000e+00],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
       [-7.1110e-01,  0.0000e+00,  1.2027e+00],
       [-2.1138e+00,  1.0000e-03,  1.2110e+00],
       [-2.8620e+00, -9.0000e-04,  2.3600e-02],
       [-4.5830e+00, -3.0000e-04,  2.3800e-02],
       [-2.1126e+00, -1.5500e-02, -1.1687e+00],
       [-7.1140e-01, -1.9400e-02, -1.1991e+00],
       [ 1.3510e-01, -3.8900e-02, -2.6974e+00],
       [ 3.5697e+00,  6.2170e-01,  2.2400e-02],
       [ 5.0817e+00, -2.6435e+00, -2.2000e-02],
       [ 1.2247e+00, -4.5958e+00, -3.5000e-03],
       [ 1.6040e-01, -2.3343e+00,  8.000

In [21]:
d['coords'][1]


array([[ 3.3937e+00, -7.6760e-01,  1.7540e+00],
       [ 3.0105e+00,  5.5100e-01,  1.6653e+00],
       [ 3.6155e+00,  1.5228e+00,  2.4663e+00],
       [ 3.2310e+00,  2.8691e+00,  2.3810e+00],
       [ 4.0361e+00,  3.9805e+00,  3.4195e+00],
       [ 2.2284e+00,  3.2991e+00,  1.4981e+00],
       [ 1.6309e+00,  2.2909e+00,  7.1720e-01],
       [ 1.9982e+00,  9.4020e-01,  7.8890e-01],
       [ 1.3760e+00,  0.0000e+00,  0.0000e+00],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
       [-7.1110e-01,  0.0000e+00,  1.2027e+00],
       [-2.1138e+00,  1.0000e-03,  1.2110e+00],
       [-2.8620e+00, -9.0000e-04,  2.3600e-02],
       [-4.5830e+00, -3.0000e-04,  2.3800e-02],
       [-2.1126e+00, -1.5500e-02, -1.1687e+00],
       [-7.1140e-01, -1.9400e-02, -1.1991e+00],
       [ 1.3510e-01, -3.8900e-02, -2.6974e+00],
       [ 2.8720e+00, -1.2980e+00,  1.1298e+00],
       [ 4.3944e+00,  1.2439e+00,  3.1722e+00],
       [ 1.9274e+00,  4.3359e+00,  1.4282e+00],
       [ 8.4260e-01,  2.5772e+00,  2.210